<a href="https://colab.research.google.com/github/swapnildahare/-AlpaCare-Medical-Instruction-Assistant-/blob/main/_AlpaCare_Medical_Instruction_Assistant__(1)_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -q git+https://github.com/huggingface/peft.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
pip install -q transformers accelerate datasets bitsandbytes safetensors evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.1 MB/s eta 0:00:00


In [ ]:
pip install -q einops

In [ ]:
from pathlib import Path
import os
import math
import random
import json
from datasets import load_dataset
import transformers
import torch

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

In [ ]:
print('Transformers', transformers.__version__)
print('Torch', torch.__version__)

Transformers 4.56.2
Torch 2.8.0+cu126


In [ ]:
ARTIFACT_DIR = Path('/content/adapters')
ARTIFACT_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
BASE_MODEL = "togethercomputer/RedPajama-INCITE-Chat-3B-v1"

MAX_LENGTH = 1024
BATCH_SIZE = 4
GRAD_ACCUM = 8
NUM_EPOCHS = 3
LEARNING_RATE = 2e-4
OUTPUT_DIR = '/content/outputs'
ADAPTER_NAME = 'alpacare-lora'
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device:', DEVICE)

Device: cuda


In [ ]:
print('Loading dataset lavita/AlpaCare-MedInstruct-52k...')
dataset = load_dataset('lavita/AlpaCare-MedInstruct-52k')

def make_prompt(example):
    instruction = example.get('instruction') or example.get('prompt') or ''
    inp = example.get('input') or ''
    output = example.get('output') or example.get('response') or example.get('answer') or ''
    if inp.strip():
        return {"prompt": f"### Instruction:\\n{instruction}\\n\\n### Input:\\n{inp}\\n\\n### Response:\\n", "response": output}
    return {"prompt": f"### Instruction:\\n{instruction}\\n\\n### Response:\\n", "response": output}

dataset['train'] = dataset['train'].map(make_prompt, remove_columns=dataset['train'].column_names)


print(dataset)
print('Example record:')
print(dataset['train'][0])

Loading dataset lavita/AlpaCare-MedInstruct-52k...


Map:   0%|          | 0/52002 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['prompt', 'response'],
        num_rows: 52002
    })
})
Example record:
{'prompt': '### Instruction:\\nExplain why a mass in the lung could cause shortness of breath.\\n\\n### Input:\\n<noinput>\\n\\n### Response:\\n', 'response': 'A mass in the lung could cause shortness of breath due to several reasons. First, the mass can physically obstruct the air passages, causing difficulty in airflow and leading to breathing difficulties. Second, if the mass is cancerous or infected, it can cause inflammation and damage to lung tissue, reducing its functional capacity and compromising normal breathing. Additionally, a lung mass can compress adjacent structures such as blood vessels, bronchi, or the diaphragm, further impeding normal respiratory function. Overall, any interference with the normal flow of air in the lungs caused by a mass can result in inadequate oxygen exchange and subsequent shortness of breath.\n\nThe answer is: A mass in 

In [ ]:
from datasets import Dataset, concatenate_datasets
# removed the redundant dataset['train'] = dataset['train'].map(...) line
dataset = {split: dataset[split].map(make_prompt, remove_columns=dataset[split].column_names) for split in dataset if split in ['train', 'test']}

Map:   0%|          | 0/52002 [00:00<?, ? examples/s]

In [ ]:
#  small curated refusal dataset to teach safe refusals
refusal_examples = [
{
'prompt': "### Instruction:\nI have chest pain and shortness of breath, what is wrong with me?\n\n\n### Response:\n",
'response': ("I’m sorry — I can’t provide medical diagnoses. I can offer general instructional information about first aid and when to contact medical services. "
"If you or someone else may be experiencing a medical emergency (difficulty breathing, severe chest pain, fainting, sudden confusion), please call your local emergency number immediately."),
},
{
'prompt': "### Instruction:\nCan you prescribe a medication for my fever?\n\n\n### Response:\n",
'response': ("I can’t prescribe medications. Medication prescriptions must come from a licensed healthcare professional after an assessment. "
"If you are unwell, please consult a doctor or pharmacist.")
}
]

In [ ]:
from datasets import Dataset, concatenate_datasets
ref_ds = Dataset.from_list(refusal_examples)
dataset['train'] = concatenate_datasets([dataset['train'], ref_ds])
print('Training size after adding refusals:', len(dataset['train']))

Training size after adding refusals: 52004


In [ ]:
print('Loading tokenizer for', BASE_MODEL)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
if tokenizer.pad_token is None:
   tokenizer.pad_token = tokenizer.eos_token

Loading tokenizer for togethercomputer/RedPajama-INCITE-Chat-3B-v1


tokenizer_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

In [ ]:
from google.colab import userdata
import os


hf_token = userdata.get('HE_token')


if hf_token is None:
    print("HE_TOKEN not found in Colab secrets. Please add your Hugging Face token as a secret named 'HE_token'.")
else:

    os.environ['HE_token'] = hf_token
    print("HE_token successfully loaded and set as environment variable.")

HE_token successfully loaded and set as environment variable.


In [ ]:
MAX_LENGTH = 512

def tokenize_fn(examples):
   input_texts = [p + r for p, r in zip(examples['prompt'], examples['response'])]
   tokenized_full = tokenizer(input_texts, truncation=True, max_length=MAX_LENGTH, padding='max_length')

   prompt_tokenized = tokenizer(examples['prompt'], truncation=True, max_length=MAX_LENGTH)
   labels = []
   for i in range(len(input_texts)):
       ids = tokenized_full['input_ids'][i].copy()
       prompt_len = len(prompt_tokenized['input_ids'][i])

       ids[:prompt_len] = [-100] * prompt_len
       labels.append(ids)

   tokenized_full['labels'] = labels
   return tokenized_full

In [ ]:
print('Tokenizing dataset...')
tokenized = {split: dataset[split].map(tokenize_fn, batched=True, remove_columns=dataset[split].column_names) for split in dataset}
print('Tokenized sample:')
print({k: tokenized['train'][0][k] for k in ['input_ids','labels']})

Tokenizing dataset...


Map:   0%|          | 0/52004 [00:00<?, ? examples/s]

Tokenized sample:
{'input_ids': [4118, 41959, 8048, 79, 4118, 41959, 8048, 79, 1672, 19104, 2139, 247, 2280, 275, 253, 6400, 812, 2847, 2159, 1255, 273, 6345, 4880, 79, 61, 79, 4118, 19832, 8048, 79, 29, 2369, 5423, 13544, 79, 61, 79, 4118, 19371, 8048, 79, 61, 79, 61, 79, 4118, 19371, 8048, 79, 34, 2280, 275, 253, 6400, 812, 2847, 2159, 1255, 273, 6345, 1955, 281, 2067, 4606, 15, 3973, 13, 253, 2280, 476, 13318, 23040, 253, 2329, 24392, 13, 8479, 10183, 275, 2329, 5449, 285, 4283, 281, 14578, 12748, 15, 6347, 13, 604, 253, 2280, 310, 2923, 528, 390, 9813, 13, 352, 476, 2847, 10696, 285, 4723, 281, 6400, 4408, 13, 8493, 697, 5164, 5350, 285, 48637, 2622, 14578, 15, 9157, 13, 247, 6400, 2280, 476, 19477, 9701, 5289, 824, 347, 2614, 12671, 13, 13605, 4635, 13, 390, 253, 34622, 13, 2007, 1607, 7022, 2622, 11900, 1159, 15, 15699, 13, 667, 11689, 342, 253, 2622, 2685, 273, 2329, 275, 253, 18926, 4269, 407, 247, 2280, 476, 906, 275, 18766, 7768, 6431, 285, 6774, 2159, 1255, 273, 6345, 15, 18

In [ ]:
print('Loading base model (8-bit) - this may take a while...')
model = AutoModelForCausalLM.from_pretrained(
BASE_MODEL,
load_in_8bit=True,
device_map='auto',
)

Loading base model (8-bit) - this may take a while...


config.json:   0%|          | 0.00/630 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


pytorch_model.bin:   0%|          | 0.00/5.69G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.69G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
model = prepare_model_for_kbit_training(model)
TARGET_MODULES = ["query_key_value"]
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=TARGET_MODULES,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 2,621,440 || all params: 2,778,485,760 || trainable%: 0.0943


In [ ]:
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTNeoXForCausalLM(
      (gpt_neox): GPTNeoXModel(
        (embed_in): Embedding(50432, 2560)
        (emb_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-31): 32 x GPTNeoXLayer(
            (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
            (post_attention_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
            (post_attention_dropout): Dropout(p=0.0, inplace=False)
            (post_mlp_dropout): Dropout(p=0.0, inplace=False)
            (attention): GPTNeoXAttention(
              (query_key_value): lora.Linear8bitLt(
                (base_layer): Linear8bitLt(in_features=2560, out_features=7680, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2560, out_features

In [ ]:
training_args = TrainingArguments(
output_dir=OUTPUT_DIR,
per_device_train_batch_size=BATCH_SIZE,
gradient_accumulation_steps=GRAD_ACCUM,
num_train_epochs=NUM_EPOCHS,
learning_rate=LEARNING_RATE,
fp16=True,
logging_steps=50,
save_total_limit=2,
remove_unused_columns=False,
report_to='none',
)

In [ ]:
trainer = Trainer(
model=model,
args=training_args,
train_dataset=tokenized['train'],
tokenizer=tokenizer,
)

/tmp/ipython-input-357569606.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
subset = dataset["train"].shuffle(seed=42).select(range(500))  # only 500 samples
tokenized_small = subset.map(tokenize_fn, batched=True, remove_columns=subset.column_names)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,             # 1 epoch
    learning_rate=2e-4,
    fp16=True,
    logging_steps=20,
    save_strategy="no",
    report_to="none",
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_small,
    tokenizer=tokenizer,
)

trainer.train()

/tmp/ipython-input-1844027803.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 0}.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr

Step,Training Loss
20,1.712200
40,0.542700
60,0.573400
80,0.415700
100,0.463000
120,0.511600


TrainOutput(global_step=125, training_loss=0.6991746253967285, metrics={'train_runtime': 582.378, 'train_samples_per_second': 0.859, 'train_steps_per_second': 0.215, 'total_flos': 4069447434240000.0, 'train_loss': 0.6991746253967285, 'epoch': 1.0})

In [ ]:
adapter_path = ARTIFACT_DIR / ADAPTER_NAME
adapter_path.mkdir(parents=True, exist_ok=True)
model.save_pretrained(adapter_path)
print('Adapter saved to', adapter_path)


Adapter saved to /content/adapters/alpacare-lora


In [ ]:
tokenizer.save_pretrained(adapter_path)

('/content/adapters/alpacare-lora/tokenizer_config.json',
 '/content/adapters/alpacare-lora/special_tokens_map.json',
 '/content/adapters/alpacare-lora/tokenizer.json')

In [ ]:
from peft import PeftModel


print('Loading base model for inference...')
base = AutoModelForCausalLM.from_pretrained(BASE_MODEL, load_in_8bit=True, device_map='auto')
model_inf = PeftModel.from_pretrained(base, adapter_path)
model_inf.eval()

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading base model for inference...


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTNeoXForCausalLM(
      (gpt_neox): GPTNeoXModel(
        (embed_in): Embedding(50432, 2560)
        (emb_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-31): 32 x GPTNeoXLayer(
            (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
            (post_attention_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
            (post_attention_dropout): Dropout(p=0.0, inplace=False)
            (post_mlp_dropout): Dropout(p=0.0, inplace=False)
            (attention): GPTNeoXAttention(
              (query_key_value): lora.Linear8bitLt(
                (base_layer): Linear8bitLt(in_features=2560, out_features=7680, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2560, out_features

In [ ]:
from transformers import GenerationConfig


def generate(prompt_text, max_new_tokens=256, do_sample=False):
   inputs = tokenizer(prompt_text, return_tensors='pt').to(model_inf.device)
   gen_cfg = GenerationConfig(max_new_tokens=max_new_tokens, do_sample=do_sample)
   with torch.no_grad():
     out = model_inf.generate(**inputs, generation_config=gen_cfg)
   text = tokenizer.decode(out[0], skip_special_tokens=True)
   return text

In [ ]:
DIAGNOSTIC_KEYWORDS = ['diagnose', 'diagnosis', "what's wrong", 'what is wrong', 'do i have', 'am i sick']
PRESCRIPTION_KEYWORDS = ['prescribe', 'prescription', 'dosage', 'mg', 'take mg', 'take this']



In [ ]:
def is_diagnostic_request(text):
   t = text.lower()
   return any(k in t for k in DIAGNOSTIC_KEYWORDS)

In [ ]:
def is_prescription_request(text):
   t = text.lower()
   return any(k in t for k in PRESCRIPTION_KEYWORDS)



In [ ]:
def safe_generate(instruction, input_text=''):
    combined = instruction + ' ' + (input_text or '')
    if is_diagnostic_request(combined) or is_prescription_request(combined):
        return ("I’m not able to provide diagnoses or prescribe medication. I can provide general instructional guidance (for example: how to clean a wound or use a medical device). "
                "If this is an emergency, please contact local emergency services.")
    prompt = f"### Instruction:\n{instruction}\n\n\n"
    if input_text:
        prompt += f"### Input:\n{input_text}\n\n\n"
    prompt += "### Response:\n"
    out = generate(prompt)
    # strip prompt prefix
    if out.startswith(prompt):
        return out[len(prompt):].strip()
    return out

In [ ]:
prompts = [
'I have chest pain and shortness of breath, what is wrong with me?',
'Can you prescribe me 500 mg of paracetamol for fever?',
'How to perform basic wound cleaning and dressing?',
'I have a rash and fever; do I need antibiotics?'
]

In [ ]:
for p in prompts:
   print('PROMPT:', p)
   print('SAFE RESPONSE:', safe_generate(p))
   print('-------------------------')

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


PROMPT: I have chest pain and shortness of breath, what is wrong with me?
SAFE RESPONSE: I’m not able to provide diagnoses or prescribe medication. I can provide general instructional guidance (for example: how to clean a wound or use a medical device). If this is an emergency, please contact local emergency services.
-------------------------
PROMPT: Can you prescribe me 500 mg of paracetamol for fever?
SAFE RESPONSE: I’m not able to provide diagnoses or prescribe medication. I can provide general instructional guidance (for example: how to clean a wound or use a medical device). If this is an emergency, please contact local emergency services.
-------------------------
PROMPT: How to perform basic wound cleaning and dressing?


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


SAFE RESPONSE: 1. Open the wound and inspect it for any signs of infection or bleeding.
2. Clean the wound thoroughly with soap and water.
3. Gently pat dry with a clean cloth.
4. Apply a sterile dressing to cover the wound.
5. Cover the dressing with a bandage to prevent further bleeding.
6. Monitor the wound regularly for signs of infection or bleeding.
7. If necessary, consult a healthcare professional for further advice.

Remember to follow these steps carefully and take appropriate precautions to prevent further complications.
-------------------------
PROMPT: I have a rash and fever; do I need antibiotics?
SAFE RESPONSE: If you have a fever and a rash, it is important to see a healthcare provider.

Antibiotics are not usually recommended for the treatment of a fever and a rash.

The decision to prescribe antibiotics should be based on a thorough evaluation of your symptoms, medical history, and physical examination.

It is important to discuss your symptoms with your healthcare p

In [ ]:
!pip install gradio -q
import gradio as gr


In [ ]:
def chat_with_alpacare(instruction):
    return safe_generate(instruction)

In [ ]:
gr.Interface(
    fn=chat_with_alpacare,
    inputs=gr.Textbox(lines=4, placeholder="Enter your medical instruction query..."),
    outputs=gr.Textbox(lines=10),
    title="🩺 AlpaCare Medical Instruction Assistant",
    description="A safe, non-diagnostic assistant fine-tuned with LoRA on AlpaCare-MedInstruct dataset."
).launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6f0e4f4fcfd14cd9db.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
